In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from seleniumbase import SB  # SeleniumBase 임포트
from bs4 import BeautifulSoup
from time import sleep
import time
import json
import pandas as pd

# 상품 상세 주소 리스트화
def crawl_product_info():
    url = "https://www.oliveyoung.co.kr/store/display/getBrandShopDetail.do?onlBrndCd=A003361"

    # SeleniumBase 사용
    with SB(uc=True, test=True, headless=False) as sb: # headless=False로 설정하여 브라우저 동작 확인
        sb.uc_open_with_reconnect(url, reconnect_time=60) # undetected_chromedriver로 페이지 열기

        # 1번 페이지 상품 정보 수집
        product_links = []
        sb.wait_for_selector('li[data-goods-idx]', timeout=10) # 상품 요소가 나타날 때까지 기다림
        li_elements = sb.find_elements(By.CSS_SELECTOR, 'li[data-goods-idx]')
        
        # 1페이지 상품 수 출력 추가
        print(f"🔎 1페이지 상품 수: {len(li_elements)}")
        
        for li in li_elements:
            a_tag = li.find_element(By.CSS_SELECTOR, 'a')
            href = a_tag.get_attribute('href')
            print(href)
            print("======================================================================")
            if href:
                product_links.append(href)

        # 2번 페이지부터 존재하는 모든 페이지 크롤링
        page_no = 2
        while True:
            print(f"\n✅ {page_no}페이지 클릭 시도")
        
            if not sb.is_element_present(f'a[data-page-no="{page_no}"]'):
                print(f"❌ {page_no}페이지 버튼이 존재하지 않음 - 크롤링 종료")
                break
        
            try:
                sb.click(f'a[data-page-no="{page_no}"]')
                sb.wait_for_element(f'strong[title="현재 페이지"]', timeout=10)
        
                current_page = sb.get_text('strong[title="현재 페이지"]')
                if current_page != str(page_no):
                    raise Exception(f"페이지 전환 실패 (현재 페이지: {current_page})")
        
                # 상품 li 다시 수집
                product_items = sb.find_elements(By.CSS_SELECTOR, 'li[data-goods-idx]')
                print(f"🔎 {page_no}페이지 상품 수: {len(product_items)}")
        
                for item in product_items:
                    a_tag = item.find_element(By.TAG_NAME, 'a')
                    href = a_tag.get_attribute("href")
                    product_links.append(href)

                page_no += 1
        
            except Exception as e:
                print(f"❌ {page_no}페이지 처리 실패: {e}")
                break

            

        # 창을 닫지 않고 대기 (SeleniumBase에서는 sb.driver 객체가 자동으로 닫힘)
        input("👉 Enter 키를 누르면 크롤링이 종료됩니다...")

        # sb.quit()은 with 블록을 벗어나면 자동으로 호출됩니다.
        return product_links

# 상품 상세 정보 크롤링 (이 함수는 현재 SeleniumBase를 잘 사용하고 있습니다)
def crawl_product_detail(list):
    product_data = []
    for url in list:
        print(url)
        with SB(uc=True, test=True, headless=False) as sb:
            sb.uc_open_with_reconnect(url, reconnect_time=60)

            html = sb.driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # Initialize result dictionary for 'capacity', 'detail', 'ingredients'
            # Since these are no longer being scraped, they will be None
            result = {
                'capacity': None,
                'detail': None,
                'ingredients': None
            }

            try:
                # 브랜드 명
                brand_name = sb.get_text("#moveBrandShop")

                # 제품명
                product_name = sb.get_text("p.prd_name")

                # 할인가
                discount_price = sb.get_text("span.price-2 strong")

                # 정가
                if sb.is_element_present("span.price-1 strike"):
                    origin_price = sb.get_text("span.price-1 strike")
                else:
                    origin_price = discount_price

                # 세일플래그
                flags = []
                span_elements = sb.find_elements("css selector", "p#icon_area span")
                for span in span_elements:
                    flags.append(span.text.strip())
                print("기본 정보 수집 성공")
            except Exception as e:
                print("기본 정보 수집 실패:", e)
                # Set default values if basic info collection fails
                brand_name = None
                product_name = None
                discount_price = None
                origin_price = None
                flags = []

            # 리뷰정보 클릭 및 수집
            totalComment = None
            numOfReviews = None
            avgReview = None
            pctOf5 = None
            pctOf4 = None
            pctOf3 = None
            pctOf2 = None
            pctOf1 = None
            review_detail = ""

            try:
                sb.click("a.goods_reputation")
                sleep(2) # 이 부분은 sb.wait_for_element_visible 등으로 대체하는 것이 좋습니다.
                print("✅ 리뷰 정보 탭 클릭 완료")
                # 리뷰정리
                totalComment = sb.get_text("div.grade_img em")
                # 리뷰갯수
                numOfReviews = sb.get_text("div.star_area em")
                # 리뷰 평점
                avgReview = sb.get_text("div.star_area strong")
                # 리뷰 점수 퍼센트
                percent_elements = sb.find_elements("css selector", "ul.graph_list span.per")
                percent_list = [el.text.strip() for el in percent_elements]
                pctOf5 = percent_list[0] if len(percent_list) > 0 else None
                pctOf4 = percent_list[1] if len(percent_list) > 1 else None
                pctOf3 = percent_list[2] if len(percent_list) > 2 else None
                pctOf2 = percent_list[3] if len(percent_list) > 3 else None
                pctOf1 = percent_list[4] if len(percent_list) > 4 else None

                # 리뷰 정보 (polls)
                review_detail = []  # 최종 json 형태 저장용
                
                polls = sb.find_elements("css selector", "dl.poll_type2.type3")
                for poll in polls:
                    try:
                        # 설문 제목 (예: 피부타입)
                        title = poll.find_element("css selector", "span").text.strip()
                
                        # 하위 항목들 (li)
                        li_tags = poll.find_elements("css selector", "ul.list > li")
                        for li in li_tags:
                            label = li.find_element("css selector", "span.txt").text.strip()
                            percent = li.find_element("css selector", "em.per").text.strip()
                
                            # 딕셔너리로 저장
                            review_detail.append({
                                "type": title,
                                "value": label,
                                "gauge": percent
                            })
                
                    except Exception as e:
                        print("리뷰 정보 오류:", e)
            except Exception as e:
                print("리뷰 정보 탭 클릭 실패:", e)
                # If review info clicking fails, ensure review-related variables are None or empty
                totalComment = None
                numOfReviews = None
                avgReview = None
                pctOf5 = None
                pctOf4 = None
                pctOf3 = None
                pctOf2 = None
                pctOf1 = None
                review_detail = ""

            # 저장
            product_info = {
                "brand": brand_name,  # 브랜드명
                "product": product_name,  # 상품이름
                "discountPrice": discount_price,  # 할인가
                "originPrice": origin_price,  # 정가
                "isPB": 1,  # Pb여부
                "flag": flags,  # 혜택
                "totalcoment": totalComment,
                "numOfReviews": numOfReviews,
                "avgReview": avgReview,
                "pctOf5": pctOf5,
                "pctOf4": pctOf4,
                "pctOf3": pctOf3,
                "pctOf2": pctOf2,
                "pctOf1": pctOf1,
                "capacity": result['capacity'],
                "detail": result['detail'],
                "ingredients": result['ingredients'],
                "reviewDetail":  json.dumps(review_detail, ensure_ascii=False)
            }
            product_data.append(product_info)
    from pprint import pprint
    pprint(product_data)
    return product_data


In [8]:
product_links = crawl_product_info()

🔎 1페이지 상품 수: 24
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000229187&dispCatNo=9000002&trackingCd=BrandA003361_PROD&t_page=%EB%B8%8C%EB%9E%9C%EB%93%9C%EA%B4%80&t_click=%EC%A0%84%EC%B2%B4%EC%83%81%ED%92%88__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=1
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000228378&dispCatNo=9000002&trackingCd=BrandA003361_PROD&t_page=%EB%B8%8C%EB%9E%9C%EB%93%9C%EA%B4%80&t_click=%EC%A0%84%EC%B2%B4%EC%83%81%ED%92%88__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=2
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000229188&dispCatNo=9000002&trackingCd=BrandA003361_PROD&t_page=%EB%B8%8C%EB%9E%9C%EB%93%9C%EA%B4%80&t_click=%EC%A0%84%EC%B2%B4%EC%83%81%ED%92%88__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=3
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000147782&dispCatNo=9000002&trackingCd=BrandA003361_PROD&t_page=%EB%B8%8C%EB%9E%9C%EB%93%9C%EA%B4%80&t_click=%EC%A0

👉 Enter 키를 누르면 크롤링이 종료됩니다... 


In [9]:
len(product_links)

47

In [ ]:
delight = crawl_product_detail(product_links)

https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000229187&dispCatNo=9000002&trackingCd=BrandA003361_PROD&t_page=%EB%B8%8C%EB%9E%9C%EB%93%9C%EA%B4%80&t_click=%EC%A0%84%EC%B2%B4%EC%83%81%ED%92%88__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=1
기본 정보 수집 성공
✅ 리뷰 정보 탭 클릭 완료
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000228378&dispCatNo=9000002&trackingCd=BrandA003361_PROD&t_page=%EB%B8%8C%EB%9E%9C%EB%93%9C%EA%B4%80&t_click=%EC%A0%84%EC%B2%B4%EC%83%81%ED%92%88__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=2
기본 정보 수집 성공
✅ 리뷰 정보 탭 클릭 완료
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000229188&dispCatNo=9000002&trackingCd=BrandA003361_PROD&t_page=%EB%B8%8C%EB%9E%9C%EB%93%9C%EA%B4%80&t_click=%EC%A0%84%EC%B2%B4%EC%83%81%ED%92%88__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=3
기본 정보 수집 성공
✅ 리뷰 정보 탭 클릭 완료
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000147782&dispCatNo=9000002&trackingCd=BrandA00